<a href="https://colab.research.google.com/github/wudsai/IBM/blob/main/IBM_Data_Science_Coursera_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SMART AI enabled event city planner**

## **Introduction**

After the pandamic is done. The will be the best events in the party capitals of europe. For tourists there will be my company which ist mangaging this events in those citys Athens, Budapest, Berlin and Prague.
The company has to put a good program, including a hotel ,a hall for meetings, 
places of landscape to visit, stores for shopping, restaurants and cafes. So this project is about taking input from a user in form of this europeen event cities and make a list of places of landscape in the selected event city, including the nearest restaurants, cafes, and shopping hangouts

## **Data Description**

The data used in this project is provided by Foursquare location data. The data are grouped by landscape area, and each area included the information about this area and all information about restaurants, cafes, and stores which in this area.

## **Table of contents**

#### 01- Libraries
#### 02- Foursquare Credentials
#### 03- Define the city and get its latitude & longitude
#### 04- Hotels search
#### 05- Parks search
#### 06- Restaurants search
#### 07- Cafeteria search
#### 08- Shopping stores search
#### 09- Map to visualize hotels, shopping stores and Cafeteria
#### 10- Map to visualize Park, Restaurant and Cafeteria

## **Import Libraries**

In [ ]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner
# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
# !pip install folium
import folium # plotting library

## **Define Foursquare Credentials**

In [ ]:
ClIENT_ID = 'IN0FVR5XYV5JDJRQVJMP3BAX0XPFOLWK3LMJ2Z4YH0ZZXLXQ' 
ClIENT_SECRET = 'YLE4XQ4ROLZFWWKVMJQABWRM5Q0VG2YX3NYPXB3HARW2JOF0' 
VERSION = '20190714'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: IN0FVR5XYV5JDJRQVJMP3BAX0XPFOLWK3LMJ2Z4YH0ZZXLXQ
Foursquare_Secret:YLE4XQ4ROLZFWWKVMJQABWRM5Q0VG2YX3NYPXB3HARW2JOF0


## **Get the city and its latitude & longitude** 

In [ ]:
# get the city and its latitude & longitude 
city = input("Choose a city:[Athens','Budapest','Berlin', 'Belgrade','Prague\n") #Though it can hancle other cities as well,but some fields might become redundant.Take care while changing
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

Choose a city:[Athens','Budapest','Berlin', 'Belgrade','Prague
Belgrade
44.8178131 20.4568974


## **Search for Hotels**

In [ ]:
# search for hotels
search_query = 'Hotel'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IN0FVR5XYV5JDJRQVJMP3BAX0XPFOLWK3LMJ2Z4YH0ZZXLXQ&client_secret=YLE4XQ4ROLZFWWKVMJQABWRM5Q0VG2YX3NYPXB3HARW2JOF0&ll=44.8178131,20.4568974&v=20190714&query=Hotel&radius=1000&limit=30'

In [ ]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4c666bd4c2ead13a1eb2b380,BAH | Belgrade Art Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1618654703,False,Knez Mihailova 27,Zmaj Jovina,44.816806,20.457969,"[{'label': 'display', 'lat': 44.81680649182447...",140,11000,RS,Београд,Central Serbia,Србија,"[Knez Mihailova 27 (Zmaj Jovina), 11000 Београ...",NaN,NaN
1,56ef27dc498e49e34e73bba4,Hotel Opera,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1618654703,False,Braće Jugović 16,Bulevar despota Stefana,44.816261,20.462143,"[{'label': 'display', 'lat': 44.81626099943074...",448,11000,RS,Београд,Central Serbia,Србија,"[Braće Jugović 16 (Bulevar despota Stefana), 1...",NaN,NaN
2,5231c9a411d2c09992015656,Hotel Srbija Garden,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1618654703,False,Brankova 13-15,btw Pop Lukina & Carice Milice,44.814636,20.455494,"[{'label': 'display', 'lat': 44.81463624632322...",370,11000,RS,Београд,Central Serbia,Србија,[Brankova 13-15 (btw Pop Lukina & Carice Milic...,81632096,NaN
3,4ddd019452b177ff2e791ef4,Hotel Royal Inn,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1618654703,False,Kralja Petra 56,Zmaja od Noćaja,44.821258,20.457135,"[{'label': 'display', 'lat': 44.82125790325615...",383,11000,RS,Београд,Central Serbia,Србија,"[Kralja Petra 56 (Zmaja od Noćaja), 11000 Беог...",NaN,NaN
4,4bd753b9304fce72656233ab,Design Hotel Mr. President,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1618654703,False,Karađorđeva 75,Lička,44.810819,20.455288,"[{'label': 'display', 'lat': 44.81081907627232...",788,11000,RS,Београд,Central Serbia,Србија,"[Karađorđeva 75 (Lička), 11000 Београд, Србија]",NaN,NaN


## **Clean Hotel Dataframe**

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,BAH | Belgrade Art Hotel,Hotel,Knez Mihailova 27,Zmaj Jovina,44.816806,20.457969,"[{'label': 'display', 'lat': 44.81680649182447...",140,11000,RS,Београд,Central Serbia,Србија,"[Knez Mihailova 27 (Zmaj Jovina), 11000 Београ...",NaN,4c666bd4c2ead13a1eb2b380
1,Hotel Opera,Hotel,Braće Jugović 16,Bulevar despota Stefana,44.816261,20.462143,"[{'label': 'display', 'lat': 44.81626099943074...",448,11000,RS,Београд,Central Serbia,Србија,"[Braće Jugović 16 (Bulevar despota Stefana), 1...",NaN,56ef27dc498e49e34e73bba4
2,Hotel Srbija Garden,Hotel,Brankova 13-15,btw Pop Lukina & Carice Milice,44.814636,20.455494,"[{'label': 'display', 'lat': 44.81463624632322...",370,11000,RS,Београд,Central Serbia,Србија,[Brankova 13-15 (btw Pop Lukina & Carice Milic...,NaN,5231c9a411d2c09992015656
3,Hotel Royal Inn,Hotel,Kralja Petra 56,Zmaja od Noćaja,44.821258,20.457135,"[{'label': 'display', 'lat': 44.82125790325615...",383,11000,RS,Београд,Central Serbia,Србија,"[Kralja Petra 56 (Zmaja od Noćaja), 11000 Беог...",NaN,4ddd019452b177ff2e791ef4
4,Design Hotel Mr. President,Hotel,Karađorđeva 75,Lička,44.810819,20.455288,"[{'label': 'display', 'lat': 44.81081907627232...",788,11000,RS,Београд,Central Serbia,Србија,"[Karađorđeva 75 (Lička), 11000 Београд, Србија]",NaN,4bd753b9304fce72656233ab


In [ ]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_dataframe

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,BAH | Belgrade Art Hotel,Hotel,Knez Mihailova 27,Zmaj Jovina,44.816806,20.457969,"[{'label': 'display', 'lat': 44.81680649182447...",140,11000,RS,Београд,Central Serbia,Србија,"[Knez Mihailova 27 (Zmaj Jovina), 11000 Београ...",NaN,4c666bd4c2ead13a1eb2b380
1,Hotel Opera,Hotel,Braće Jugović 16,Bulevar despota Stefana,44.816261,20.462143,"[{'label': 'display', 'lat': 44.81626099943074...",448,11000,RS,Београд,Central Serbia,Србија,"[Braće Jugović 16 (Bulevar despota Stefana), 1...",NaN,56ef27dc498e49e34e73bba4
2,Hotel Srbija Garden,Hotel,Brankova 13-15,btw Pop Lukina & Carice Milice,44.814636,20.455494,"[{'label': 'display', 'lat': 44.81463624632322...",370,11000,RS,Београд,Central Serbia,Србија,[Brankova 13-15 (btw Pop Lukina & Carice Milic...,NaN,5231c9a411d2c09992015656
3,Hotel Royal Inn,Hotel,Kralja Petra 56,Zmaja od Noćaja,44.821258,20.457135,"[{'label': 'display', 'lat': 44.82125790325615...",383,11000,RS,Београд,Central Serbia,Србија,"[Kralja Petra 56 (Zmaja od Noćaja), 11000 Беог...",NaN,4ddd019452b177ff2e791ef4
4,Design Hotel Mr. President,Hotel,Karađorđeva 75,Lička,44.810819,20.455288,"[{'label': 'display', 'lat': 44.81081907627232...",788,11000,RS,Београд,Central Serbia,Србија,"[Karađorđeva 75 (Lička), 11000 Београд, Србија]",NaN,4bd753b9304fce72656233ab
5,Hotel Pavilion,Hotel,Dobračina 39,Cara Dušana,44.820659,20.463947,"[{'label': 'display', 'lat': 44.820659, 'lng':...",640,11158,RS,Belgrade,Grad Beograd,Србија,"[Dobračina 39 (Cara Dušana), 11158 Belgrade, С...",NaN,5f173863c32bef35d74d3508
6,Hotel Muzeum,Hotel,Čika Ljubina 3-5,NaN,44.816522,20.459678,"[{'label': 'display', 'lat': 44.816522, 'lng':...",262,11000,RS,Belgrade,Grad Beograd,Србија,"[Čika Ljubina 3-5, 11000 Belgrade, Србија]",NaN,5c5be33635f983002c029dee
7,Hotel Amsterdam,Hotel,Jug Bogdanova 10,NaN,44.814109,20.455853,"[{'label': 'display', 'lat': 44.814109, 'lng':...",420,11000,RS,Belgrade,Grad Beograd,Србија,"[Jug Bogdanova 10, 11000 Belgrade, Србија]",NaN,5c1cea792619ee002c353ca7
8,Hotel Kalemegdan Park 1899,Hotel,Pariska 15,Kneza Mihaila,44.819553,20.452954,"[{'label': 'display', 'lat': 44.8195525428506,...",366,11000,RS,Београд,Central Serbia,Србија,"[Pariska 15 (Kneza Mihaila), 11000 Београд, Ср...",NaN,4e4e9d3662e14b77e3900aaa
9,Hotel EVROPA Spa Centar,Spa,Sremska 1,NaN,44.814807,20.459950,"[{'label': 'display', 'lat': 44.814807, 'lng':...",412,NaN,RS,Београд,Central Serbia,Србија,"[Sremska 1, Београд, Србија]",NaN,557fe9a7498eca6e4d854880


In [ ]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state,neighborhood
11,Hotel Evropa,Hotel,Sremska 1,44.814779,20.459986,11000,Central Serbia,Trg republike


In [ ]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state,neighborhood
11,Hotel Evropa,Hotel,Sremska 1,44.814779,20.459986,11000,Central Serbia,Trg republike


In [ ]:
# delete rows which has duplicate hotel's name
df_hotel = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotel

,name,categories,address,lat,lng,postalCode,state,neighborhood
11,Hotel Evropa,Hotel,Sremska 1,44.814779,20.459986,11000,Central Serbia,Trg republike


## **Search for Parks**

In [ ]:
# search for Parks
search_query = 'Park'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IN0FVR5XYV5JDJRQVJMP3BAX0XPFOLWK3LMJ2Z4YH0ZZXLXQ&client_secret=YLE4XQ4ROLZFWWKVMJQABWRM5Q0VG2YX3NYPXB3HARW2JOF0&ll=44.8178131,20.4568974&v=20190714&query=Park&radius=10000&limit=30'

In [ ]:
# Send the GET Request and examine the results
presults = requests.get(url).json()
#presults

In [ ]:
# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
park_dataframe = json_normalize(venues)
park_dataframe.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4ba2a88bf964a520640d38e3,Tašmajdanski Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1618654713,False,Bulevar kralja Aleksandra,Beogradska,44.809697,20.470083,"[{'label': 'display', 'lat': 44.80969685360424...",1378,11000,RS,Beograd,Vračar,Србија,"[Bulevar kralja Aleksandra (Beogradska), 11000...",NaN
1,531229f7498e96f5b85462b5,Калемегдански парк | Kalemegdanski Park (Калем...,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1618654713,False,Kalemegdan,Kalemegdan,44.823293,20.450857,"[{'label': 'display', 'lat': 44.82329287797476...",774,11000,RS,Dorćol,Central Serbia,Србија,"[Kalemegdan (Kalemegdan), 11000 Dorćol, Србија]","Kalemegdan, Belgrade"
2,4cb0ee74cbab236aa6e2a473,Park Vojvode Vuka,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1618654713,False,Topličin Venac,Carice Milice,44.816639,20.455016,"[{'label': 'display', 'lat': 44.81663920702975...",197,11000,RS,Београд,Central Serbia,Србија,"[Topličin Venac (Carice Milice), 11000 Београд...",NaN
3,4b87ba91f964a520f7c831e3,Studentski park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1618654713,False,Studentski trg bb,Braće Jugovića,44.819480,20.457957,"[{'label': 'display', 'lat': 44.81947972111098...",203,11000,RS,Београд,Central Serbia,Србија,"[Studentski trg bb (Braće Jugovića), 11000 Бео...",NaN
4,4e53c9ebd1641f4f818a3a8a,Pančićev park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1618654713,False,Pančićeva,btwn Kapetan Mišina and Pančićeva,44.824984,20.463497,"[{'label': 'display', 'lat': 44.82498434524746...",953,11000,RS,Београд,Central Serbia,Србија,[Pančićeva (btwn Kapetan Mišina and Pančićeva)...,Dorćol


## **Clean Park Dataframe**

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Tašmajdanski Park,Park,Bulevar kralja Aleksandra,Beogradska,44.809697,20.470083,"[{'label': 'display', 'lat': 44.80969685360424...",1378,11000,RS,Beograd,Vračar,Србија,"[Bulevar kralja Aleksandra (Beogradska), 11000...",NaN,4ba2a88bf964a520640d38e3
1,Калемегдански парк | Kalemegdanski Park (Калем...,Park,Kalemegdan,Kalemegdan,44.823293,20.450857,"[{'label': 'display', 'lat': 44.82329287797476...",774,11000,RS,Dorćol,Central Serbia,Србија,"[Kalemegdan (Kalemegdan), 11000 Dorćol, Србија]","Kalemegdan, Belgrade",531229f7498e96f5b85462b5
2,Park Vojvode Vuka,Park,Topličin Venac,Carice Milice,44.816639,20.455016,"[{'label': 'display', 'lat': 44.81663920702975...",197,11000,RS,Београд,Central Serbia,Србија,"[Topličin Venac (Carice Milice), 11000 Београд...",NaN,4cb0ee74cbab236aa6e2a473
3,Studentski park,Park,Studentski trg bb,Braće Jugovića,44.819480,20.457957,"[{'label': 'display', 'lat': 44.81947972111098...",203,11000,RS,Београд,Central Serbia,Србија,"[Studentski trg bb (Braće Jugovića), 11000 Бео...",NaN,4b87ba91f964a520f7c831e3
4,Pančićev park,Park,Pančićeva,btwn Kapetan Mišina and Pančićeva,44.824984,20.463497,"[{'label': 'display', 'lat': 44.82498434524746...",953,11000,RS,Београд,Central Serbia,Србија,[Pančićeva (btwn Kapetan Mišina and Pančićeva)...,Dorćol,4e53c9ebd1641f4f818a3a8a


In [ ]:
# delete unnecessary columns
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,crossStreet,lat,lng,postalCode,state,neighborhood
0,Tašmajdanski Park,Park,Bulevar kralja Aleksandra,Beogradska,44.809697,20.470083,11000,Vračar,NaN
1,Калемегдански парк | Kalemegdanski Park (Калем...,Park,Kalemegdan,Kalemegdan,44.823293,20.450857,11000,Central Serbia,"Kalemegdan, Belgrade"
2,Park Vojvode Vuka,Park,Topličin Venac,Carice Milice,44.816639,20.455016,11000,Central Serbia,NaN
3,Studentski park,Park,Studentski trg bb,Braće Jugovića,44.819480,20.457957,11000,Central Serbia,NaN
4,Pančićev park,Park,Pančićeva,btwn Kapetan Mišina and Pančićeva,44.824984,20.463497,11000,Central Serbia,Dorćol
5,Pionirski park,Park,Dragoslava Jovanovića,NaN,44.810631,20.464589,11000,Central Serbia,NaN
6,Luna park Kalemegdan,Theme Park,NaN,NaN,44.824729,20.454625,NaN,Central Serbia,Kalemegdan
7,Park za pse „Jalije”,Dog Run,Pančićeva,Dunavska,44.825434,20.464703,11000,Central Serbia,"Dorćol, Belgrade"
8,Park 7,Hotel,NaN,NaN,44.819955,20.459444,NaN,Central Serbia,NaN
9,Park Nesvrstanih zemalja,Park,Brankova,NaN,44.815515,20.452294,NaN,NaN,NaN


In [ ]:
# delete rows with none values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,address,crossStreet,lat,lng,postalCode,state,neighborhood
1,Калемегдански парк | Kalemegdanski Park (Калем...,Park,Kalemegdan,Kalemegdan,44.823293,20.450857,11000,Central Serbia,"Kalemegdan, Belgrade"
4,Pančićev park,Park,Pančićeva,btwn Kapetan Mišina and Pančićeva,44.824984,20.463497,11000,Central Serbia,Dorćol
7,Park za pse „Jalije”,Dog Run,Pančićeva,Dunavska,44.825434,20.464703,11000,Central Serbia,"Dorćol, Belgrade"
22,Devojački park,Park,Kralja Milana bb,Kosovke devojke,44.809954,20.461797,11040,Central Serbia,"Savski Venac, Belgrade"
24,Park,Hotel,Njegoševa 4,Svetozara Markovića,44.805308,20.466286,11000,Central Serbia,Savski


In [ ]:
# delete rows which its category is not Park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park

,name,categories,address,crossStreet,lat,lng,postalCode,state,neighborhood
1,Калемегдански парк | Kalemegdanski Park (Калем...,Park,Kalemegdan,Kalemegdan,44.823293,20.450857,11000,Central Serbia,"Kalemegdan, Belgrade"
4,Pančićev park,Park,Pančićeva,btwn Kapetan Mišina and Pančićeva,44.824984,20.463497,11000,Central Serbia,Dorćol
22,Devojački park,Park,Kralja Milana bb,Kosovke devojke,44.809954,20.461797,11040,Central Serbia,"Savski Venac, Belgrade"


## **Search for Restaurants**

In [ ]:
# search for Restaurants
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IN0FVR5XYV5JDJRQVJMP3BAX0XPFOLWK3LMJ2Z4YH0ZZXLXQ&client_secret=YLE4XQ4ROLZFWWKVMJQABWRM5Q0VG2YX3NYPXB3HARW2JOF0&ll=44.8178131,20.4568974&v=20190714&query=Restaurant&radius=10000&limit=30'

In [ ]:
# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

In [ ]:
# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet,location.neighborhood
0,5a518c451108ba4e2138785f,Zuwar Belgrade Halal Restaurant,"[{'id': '52e81612bcbc57f1066b79ff', 'name': 'H...",v-1618654715,False,Đure Jakšića 3,44.817450,20.456618,"[{'label': 'display', 'lat': 44.81744978208313...",46,11000,RS,Београд,Central Serbia,Србија,"[Đure Jakšića 3, 11000 Београд, Србија]",483947018,NaN,NaN
1,516d5ccce4b0b80c9c23fd40,Martinović express restaurant,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",v-1618654715,False,Obilicev Venac 4,44.816522,20.456749,"[{'label': 'display', 'lat': 44.81652171521065...",144,NaN,RS,Београд,Central Serbia,Србија,"[Obilicev Venac 4, Београд, Србија]",NaN,NaN,NaN
2,5e4d80e78f54b10008da9f5c,Mama Shelter Restaurant Belgrade,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1618654715,False,Kneza Mihaila 54,44.819913,20.453682,"[{'label': 'display', 'lat': 44.819913, 'lng':...",345,11000,RS,Београд,Central Serbia,Србија,"[Kneza Mihaila 54, 11000 Београд, Србија]",NaN,NaN,NaN
3,5086bb72e4b0aafd985c1ccc,WFF Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1618654715,False,Carice Milice 15,44.816002,20.455961,"[{'label': 'display', 'lat': 44.8160019864492,...",214,11000,RS,Београд,Central Serbia,Србија,"[Carice Milice 15, 11000 Београд, Србија]",NaN,NaN,NaN
4,5bdef3c16336be002c6f512e,Turkish Restaurant Turkuaz,"[{'id': '4f04af1f2fb6e1c99f3db0bb', 'name': 'T...",v-1618654715,False,"Dositejeva 1A, Beograd 11000",44.817143,20.460706,"[{'label': 'display', 'lat': 44.81714306519704...",309,11000,RS,Beograd,Stari Grad,Србија,"[Dositejeva 1A, Beograd 11000 (Vase Čarapića),...",NaN,Vase Čarapića,NaN


## **Clean Restaurant Dataframe**

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Zuwar Belgrade Halal Restaurant,Halal Restaurant,Đure Jakšića 3,44.817450,20.456618,"[{'label': 'display', 'lat': 44.81744978208313...",46,11000,RS,Београд,Central Serbia,Србија,"[Đure Jakšića 3, 11000 Београд, Србија]",NaN,NaN,5a518c451108ba4e2138785f
1,Martinović express restaurant,Diner,Obilicev Venac 4,44.816522,20.456749,"[{'label': 'display', 'lat': 44.81652171521065...",144,NaN,RS,Београд,Central Serbia,Србија,"[Obilicev Venac 4, Београд, Србија]",NaN,NaN,516d5ccce4b0b80c9c23fd40
2,Mama Shelter Restaurant Belgrade,Restaurant,Kneza Mihaila 54,44.819913,20.453682,"[{'label': 'display', 'lat': 44.819913, 'lng':...",345,11000,RS,Београд,Central Serbia,Србија,"[Kneza Mihaila 54, 11000 Београд, Србија]",NaN,NaN,5e4d80e78f54b10008da9f5c
3,WFF Restaurant,Chinese Restaurant,Carice Milice 15,44.816002,20.455961,"[{'label': 'display', 'lat': 44.8160019864492,...",214,11000,RS,Београд,Central Serbia,Србија,"[Carice Milice 15, 11000 Београд, Србија]",NaN,NaN,5086bb72e4b0aafd985c1ccc
4,Turkish Restaurant Turkuaz,Turkish Restaurant,"Dositejeva 1A, Beograd 11000",44.817143,20.460706,"[{'label': 'display', 'lat': 44.81714306519704...",309,11000,RS,Beograd,Stari Grad,Србија,"[Dositejeva 1A, Beograd 11000 (Vase Čarapića),...",Vase Čarapića,NaN,5bdef3c16336be002c6f512e


In [ ]:
# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state,neighborhood
0,Zuwar Belgrade Halal Restaurant,Halal Restaurant,Đure Jakšića 3,44.817450,20.456618,11000,Central Serbia,NaN
1,Martinović express restaurant,Diner,Obilicev Venac 4,44.816522,20.456749,NaN,Central Serbia,NaN
2,Mama Shelter Restaurant Belgrade,Restaurant,Kneza Mihaila 54,44.819913,20.453682,11000,Central Serbia,NaN
3,WFF Restaurant,Chinese Restaurant,Carice Milice 15,44.816002,20.455961,11000,Central Serbia,NaN
4,Turkish Restaurant Turkuaz,Turkish Restaurant,"Dositejeva 1A, Beograd 11000",44.817143,20.460706,11000,Stari Grad,NaN
5,No.12 Restaurant,Steakhouse,rajićeva 12,44.819496,20.453620,NaN,Central Serbia,NaN
6,The Two Deer Restaurant,Bagel Shop,NaN,44.819048,20.462163,NaN,NaN,NaN
7,kungfu restaurant 功夫餐厅,Chinese Restaurant,"simina 33,beograd,serbia",44.817555,20.462819,NaN,Central Serbia,NaN
8,Ustanička Bar & Restaurant,Restaurant,Ustanička 66,44.785844,20.481320,11010,Central Serbia,Šumice
9,East&west sushi restaurant,Sushi Restaurant,Gospodar Jovanova 46,44.819848,20.462636,NaN,Central Serbia,NaN


In [ ]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state,neighborhood
8,Ustanička Bar & Restaurant,Restaurant,Ustanička 66,44.785844,20.481320,11010,Central Serbia,Šumice
27,Tchaikovsky,Restaurant,Terazije 20,44.813201,20.460706,11000,Central Serbia,"Terazije, Belgrade"


## **Search for Cafeteria**

In [ ]:
# search for Cafeteria
search_query = 'Cafeteria'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IN0FVR5XYV5JDJRQVJMP3BAX0XPFOLWK3LMJ2Z4YH0ZZXLXQ&client_secret=YLE4XQ4ROLZFWWKVMJQABWRM5Q0VG2YX3NYPXB3HARW2JOF0&ll=44.8178131,20.4568974&v=20190714&query=Cafeteria&radius=10000&limit=30'

In [ ]:
# Send the GET Request and examine the results
cresults = requests.get(url).json()
#cresults

In [ ]:
# assign relevant part of JSON to venues
venues = cresults['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood
0,573a5f6dcd10b4da02c93413,Espressino Cafeteria,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1618654716,False,Ulica Cara Lazara 19,44.817130,20.455451,"[{'label': 'display', 'lat': 44.81713, 'lng': ...",137,11000,RS,Belgrade,Grad Beograd,Србија,"[Ulica Cara Lazara 19, 11000 Belgrade, Србија]",NaN,NaN
1,56d09b7ccd105c8869a537c5,Cafetería Gardoš,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1618654716,False,Njegoševa 53,44.847964,20.412989,"[{'label': 'display', 'lat': 44.8479644206236,...",4824,11080,RS,Београд,Central Serbia,Србија,"[Njegoševa 53 (Kej oslobođenja), 11080 Београд...",Kej oslobođenja,NaN
2,5844a64f9850a03d83f1bfc0,Cafeteria Italiana,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1618654716,False,Uroša Martinovića 17,44.805964,20.400704,"[{'label': 'display', 'lat': 44.80596377105667...",4629,11070,RS,Београд,Central Serbia,Србија,"[Uroša Martinovića 17 (Omladinskih brigada), 1...",Omladinskih brigada,NaN
3,4bb1c5e8f964a52062a23ce3,Moment,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1618654716,False,Topolska 16,44.799687,20.477417,"[{'label': 'display', 'lat': 44.79968685750769...",2588,11000,RS,Београд,Central Serbia,Србија,"[Topolska 16 (Petrogradska), 11000 Београд, Ср...",Petrogradska,NaN
4,4d29ce2a342d6dcb0e9406cb,Jarbol,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1618654716,False,22. oktobra 14,44.838212,20.414099,"[{'label': 'display', 'lat': 44.83821235767966...",4071,11080,RS,Београд,Central Serbia,Србија,"[22. oktobra 14 (Starca Vujadina), 11080 Беогр...",Starca Vujadina,Zemun


## **Clean Cafeteria Dataframe**

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Espressino Cafeteria,Cafeteria,Ulica Cara Lazara 19,44.817130,20.455451,"[{'label': 'display', 'lat': 44.81713, 'lng': ...",137,11000,RS,Belgrade,Grad Beograd,Србија,"[Ulica Cara Lazara 19, 11000 Belgrade, Србија]",NaN,NaN,573a5f6dcd10b4da02c93413
1,Cafetería Gardoš,Coffee Shop,Njegoševa 53,44.847964,20.412989,"[{'label': 'display', 'lat': 44.8479644206236,...",4824,11080,RS,Београд,Central Serbia,Србија,"[Njegoševa 53 (Kej oslobođenja), 11080 Београд...",Kej oslobođenja,NaN,56d09b7ccd105c8869a537c5
2,Cafeteria Italiana,Coffee Shop,Uroša Martinovića 17,44.805964,20.400704,"[{'label': 'display', 'lat': 44.80596377105667...",4629,11070,RS,Београд,Central Serbia,Србија,"[Uroša Martinovića 17 (Omladinskih brigada), 1...",Omladinskih brigada,NaN,5844a64f9850a03d83f1bfc0
3,Moment,Café,Topolska 16,44.799687,20.477417,"[{'label': 'display', 'lat': 44.79968685750769...",2588,11000,RS,Београд,Central Serbia,Србија,"[Topolska 16 (Petrogradska), 11000 Београд, Ср...",Petrogradska,NaN,4bb1c5e8f964a52062a23ce3
4,Jarbol,Café,22. oktobra 14,44.838212,20.414099,"[{'label': 'display', 'lat': 44.83821235767966...",4071,11080,RS,Београд,Central Serbia,Србија,"[22. oktobra 14 (Starca Vujadina), 11080 Беогр...",Starca Vujadina,Zemun,4d29ce2a342d6dcb0e9406cb


In [ ]:
# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2

,name,categories,address,lat,lng,postalCode,state,neighborhood
0,Espressino Cafeteria,Cafeteria,Ulica Cara Lazara 19,44.817130,20.455451,11000,Grad Beograd,NaN
1,Cafetería Gardoš,Coffee Shop,Njegoševa 53,44.847964,20.412989,11080,Central Serbia,NaN
2,Cafeteria Italiana,Coffee Shop,Uroša Martinovića 17,44.805964,20.400704,11070,Central Serbia,NaN
3,Moment,Café,Topolska 16,44.799687,20.477417,11000,Central Serbia,NaN
4,Jarbol,Café,22. oktobra 14,44.838212,20.414099,11080,Central Serbia,Zemun
5,Gogolj,Coffee Shop,Požeška 142b,44.774933,20.412583,11030,Central Serbia,NaN
6,Semlin art,Restaurant,Fruškogorska 7,44.847081,20.412555,11180,Central Serbia,NaN


In [ ]:
# delete rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,postalCode,state,neighborhood
4,Jarbol,Café,22. oktobra 14,44.838212,20.414099,11080,Central Serbia,Zemun


## **Search for Shopping Stores**

In [ ]:
# search for Shopping
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IN0FVR5XYV5JDJRQVJMP3BAX0XPFOLWK3LMJ2Z4YH0ZZXLXQ&client_secret=YLE4XQ4ROLZFWWKVMJQABWRM5Q0VG2YX3NYPXB3HARW2JOF0&ll=44.8178131,20.4568974&v=20190714&query=Shopping&radius=1000&limit=30'

In [ ]:
# Send the GET Request and examine the results
sresults = requests.get(url).json()
#sresults

In [ ]:
# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,59a183936a8d862d2f09b55c,Rajićeva Shopping Center,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1618654717,False,Knez Mihailova 54,Rajićeva,44.820013,20.454109,"[{'label': 'display', 'lat': 44.82001274784061...",329,11000,RS,Београд,Central Serbia,Србија,"[Knez Mihailova 54 (Rajićeva), 11000 Београд, ..."
1,5555fec2498ed460578ed076,Rajićeva,"[{'id': '4bf58dd8d48988d1f9931735', 'name': 'R...",v-1618654717,False,Rajićeva,btw Uzun Mirkova and Gračanička,44.819106,20.455909,"[{'label': 'display', 'lat': 44.8191064339701,...",163,11000,RS,Београд,Central Serbia,Србија,"[Rajićeva (btw Uzun Mirkova and Gračanička), 1..."


## **Clean Shopping Dataframe**

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Rajićeva Shopping Center,Shopping Mall,Knez Mihailova 54,Rajićeva,44.820013,20.454109,"[{'label': 'display', 'lat': 44.82001274784061...",329,11000,RS,Београд,Central Serbia,Србија,"[Knez Mihailova 54 (Rajićeva), 11000 Београд, ...",59a183936a8d862d2f09b55c
1,Rajićeva,Road,Rajićeva,btw Uzun Mirkova and Gračanička,44.819106,20.455909,"[{'label': 'display', 'lat': 44.8191064339701,...",163,11000,RS,Београд,Central Serbia,Србија,"[Rajićeva (btw Uzun Mirkova and Gračanička), 1...",5555fec2498ed460578ed076


In [ ]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,crossStreet,lat,lng,postalCode,state
0,Rajićeva Shopping Center,Shopping Mall,Knez Mihailova 54,Rajićeva,44.820013,20.454109,11000,Central Serbia
1,Rajićeva,Road,Rajićeva,btw Uzun Mirkova and Gračanička,44.819106,20.455909,11000,Central Serbia


In [ ]:
# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2
df_Shopping

,name,categories,address,crossStreet,lat,lng,postalCode,state
0,Rajićeva Shopping Center,Shopping Mall,Knez Mihailova 54,Rajićeva,44.820013,20.454109,11000,Central Serbia
1,Rajićeva,Road,Rajićeva,btw Uzun Mirkova and Gračanička,44.819106,20.455909,11000,Central Serbia


## **Generate map to visualize hotels, shopping stores and Cafeteria and how they cluster together**

In [ ]:
# create dataframe of hotels, shopping stores and Cafeteria
hotel_neighbourhood_df = pd.concat([df_hotel, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_neighbourhood_df

,name,categories,address,lat,lng,postalCode,state,neighborhood,crossStreet
0,Hotel Evropa,Hotel,Sremska 1,44.814779,20.459986,11000,Central Serbia,Trg republike,NaN
1,Jarbol,Café,22. oktobra 14,44.838212,20.414099,11080,Central Serbia,Zemun,NaN
2,Rajićeva Shopping Center,Shopping Mall,Knez Mihailova 54,44.820013,20.454109,11000,Central Serbia,NaN,Rajićeva
3,Rajićeva,Road,Rajićeva,44.819106,20.455909,11000,Central Serbia,NaN,btw Uzun Mirkova and Gračanička


In [ ]:
# Generate map to visualize hotel neighbourhood including shopping stores and Cafeteria 
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

## **Generate map to visualize Park, Restaurant and Cafeteria and how they cluster together**

In [ ]:
# create dataframe of Park, Restaurant and Cafeteria
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
park_neighbourhood_df

,name,categories,address,crossStreet,lat,lng,postalCode,state,neighborhood
0,Калемегдански парк | Kalemegdanski Park (Калем...,Park,Kalemegdan,Kalemegdan,44.823293,20.450857,11000,Central Serbia,"Kalemegdan, Belgrade"
1,Pančićev park,Park,Pančićeva,btwn Kapetan Mišina and Pančićeva,44.824984,20.463497,11000,Central Serbia,Dorćol
2,Devojački park,Park,Kralja Milana bb,Kosovke devojke,44.809954,20.461797,11040,Central Serbia,"Savski Venac, Belgrade"
3,Ustanička Bar & Restaurant,Restaurant,Ustanička 66,NaN,44.785844,20.481320,11010,Central Serbia,Šumice
4,Tchaikovsky,Restaurant,Terazije 20,NaN,44.813201,20.460706,11000,Central Serbia,"Terazije, Belgrade"
5,Jarbol,Café,22. oktobra 14,NaN,44.838212,20.414099,11080,Central Serbia,Zemun


In [ ]:
# Generate map to visualize park neighbourhood including Restaurant and Cafeteria 
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    
park_map